In [ ]:
!pip install newsapi-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# get list of symbols available

symbols = list()
list_loc = r"/content/drive/MyDrive/STOCKS.txt"
with open(list_loc, 'r') as f:
  for line in f.readlines():
    symbols.append(line.strip())

print(len(symbols), symbols[0], symbols[len(symbols)-1])

18 INFY COALINDIA


In [ ]:
NO_OF_CHARS = 256
def badCharHeuristic(string, size):
    # Initialize all occurrence as -1
    badChar = [-1]*NO_OF_CHARS
    # Fill the actual value of last occurrence
    for i in range(size):
        badChar[ord(string[i])] = i;

    return badChar

def search(txt, pat):
    m = len(pat)
    n = len(txt)

    # create the bad character list by calling
    # the preprocessing function badCharHeuristic()
    badChar = badCharHeuristic(pat, m)

    # s is shift of the pattern with respect to text
    s = 0
    while(s <= n-m):
        j = m - 1

        # keep reducing index j of pattern while characters of pattern and text are matching at this shift s
        while j >= 0 and pat[j] == txt[s+j]:
            j -= 1

        # if pattern is present at current shift, then index j will become -1 after the above loop
        if j < 0:
            return s
        else:
            s += max(1, j - badChar[ord(txt[s+j])])

    return -1
txt = "./markets/indexes"
pat = "./"
if search(txt, pat) > -1:
    print('Found')


Found


In [ ]:
from collections import Counter
def check_service_links(a_link):
    SL = [
        'https://www.google.com/intl/',
        'https://accounts.google.com/Service',
        'https://support.google.com/',
        'https://myaccount.google.com/',
        'https://www.google.com/maps/',
        'https://www.google.com/search',
        'https://m.economictimes.com/prime'
    ]
    links = list()
    pos = 0
    for link in a_link:
        for sl in SL:
            pos = search(link, sl)
            if pos != -1:
                break;
        if pos == -1:
            links.append(link)
    # using Counter() + keys() to remove duplicated from list
    temp = Counter(links)
    links = [*temp]
    '''    print('All cleaned values. -----------------------------------------------')
    for link in links:
        print(link)
    print('-------------------------------------------------------------------')'''
    return links

def get_actual_link(links):
    # remove easily identified links
    a_link = []
    for link in links:
        if link is not None:
            if link[0] != '.' and link[0] != '/':
                a_link.append(link)
    # remove service links
    '''    print('After removing ./ and None values. --------------------------------')
    for link in a_link:
        print(link)
    print('-------------------------------------------------------------------')'''
    return check_service_links(a_link)

In [ ]:
def get_url(name):
    return 'https://www.google.com/finance/quote/'+name+':NSE?hl=en'
import requests
from bs4 import BeautifulSoup

def get_news_links(name):
    # search query : https://www.google.com/search?q=wipro+in+news+today
    r = requests.get(get_url(name))

    links = []
    soup = BeautifulSoup(r.text, 'html.parser')
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    return get_actual_link(links)
'''
for link in links:
    print(link)'''

'\nfor link in links:\n    print(link)'

In [ ]:
def get_news(links):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    news = str()
    for link in links:

        r = requests.get(link, headers=headers)

        soup = BeautifulSoup(r.text, 'html.parser')
        for link in soup.find_all('p'):
            news += link.get_text().strip()
    return news

In [5]:
!pip install yfinance

In [6]:
# Basis information by ticker name

import yfinance as yf

def get_fundamental_info(ticker_name):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    ticker_name: A SYMBOL.

  Returns:
    info: A dict with basic information about the given symbol.
    hist: historical market data of period 1 day and interval of 1 min
  """
  # get basic data
  ticker = yf.Ticker(ticker_name)

  # get historical market data
  hist = ticker.history(period="max", interval="1m")

  return ticker.info, hist

In [11]:
get_fundamental_info('MSFT')

HTTPError: ignored